Fabian Brock
Case 5
i6248959

In [1]:
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LinearRegression

# Asian Option

In [2]:
# create a method for the monte carlo pricing of asian
def MC_price_asian_option(S0, r, sigma, T, K, M, n):
    dt = T/n  # Time step
    # gbm
    St = S0 * np.exp(np.cumsum((r - 0.5 * sigma**2) * dt
                               + sigma * np.random.normal(0, np.sqrt(dt), (M, n)), axis=1))
    # avg, set n = 10 so it will be avg of 10
    average_St = np.mean(St, axis=1)

    # payoff
    payoffs = np.maximum(average_St - K, 0)

    # discounting
    discounted_payoff = np.exp(-r * T) * payoffs

    # mean
    price = np.mean(discounted_payoff)
    return np.round(price, 4)

In [3]:
S0 = 100
r = 0.02
sigma = 0.15
T = 10
K = 100
M = 100000  # sims
n = 10  # timesteps

# Calculate the Asian option price using Monte Carlo simulation
asian_option_price = MC_price_asian_option(S0, r, sigma, T, K, M, n)
print("Asian option price: ", asian_option_price)

Asian option price:  15.6967


In [4]:
from utils import MC_price # code from week 3
print("European Call price: ", MC_price(S0, r, sigma, T, K, M, n))

European Call price:  27.561


## As expected the asian option is cheaper

In [5]:
from utils import black_scholes_call_price # BSM code from previous weeks
def compute_moments(S0, r, sigma, T, n):
    dt = T / n
    # first moment for each timstep
    # S0 * e^(r * j)
    first_moment_terms = S0 * np.exp(r * (np.arange(1, n + 1) * dt))
    first_moment = np.mean(first_moment_terms)

    # 2nd monent for each pair timesteps
    # S0^2 * e^(r * (i + j) * dt + sigma^2 * min(i, j) * dt)
    second_moment_sum = 0
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            second_moment_sum += S0**2 * np.exp(r * (i + j) * dt + sigma**2 * min(i, j) * dt)

    second_moment = second_moment_sum  / n**2 # (1/10)^2 since n = 10

    return first_moment, second_moment

def compute_log_normal_approximation_params(first_moment, second_moment, r, T):
    A0 = first_moment * np.exp(-r * T)
    b_squared = (np.log(second_moment) - 2 * np.log(A0) - 2 * r * T) / T
    b = np.sqrt(b_squared)
    return A0, b

# calc moments of A
first_moment, second_moment = compute_moments(S0, r, sigma, T, n)

# calc params for log normal apx
A0, b = compute_log_normal_approximation_params(first_moment, second_moment, r, T)

# BSM for log normal apx params
closed_form_price = black_scholes_call_price(A0, K, r, b, T)

print("A0: ", A0)
print("b: ", b)
# monte carlo asian price
asian_option_price = MC_price_asian_option(S0, r, sigma, T, K, M, n)

print("Closed-form price: ", closed_form_price)
print("Asian option price: ", asian_option_price)

A0:  91.5439908295937
b:  0.09584030077452006
Closed-form price:  15.973823843043995
Asian option price:  15.7333


## The closed form price is very close to the monte carlo price

# Unit Linked with continuous guarantee

In [6]:
def binomial_tree_option_price(S0, K, r, n, T, sigma):
    dt = T / n  # Time step
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    # init tree
    tree = np.zeros((n+1, n+1))

    # final nodes values
    for i in range(n+1):
        tree[i, n] = max(K, S0 * u**(n-i) * d**i)

    # backwards get optoin value
    for j in range(n-1, -1, -1):
        for i in range(j+1):
            # max of hold or exercise
            hold = np.exp(-r * dt) * (p * tree[i, j+1] + (1-p) * tree[i+1, j+1])
            exercise = 100
            tree[i, j] = max(hold, exercise)

    return tree[0, 0]

In [7]:
def LSMC_unit_linked_price(S0, K, r, T, sigma, M, n):
    dt = T / n  
    discount_factor = np.exp(-r * dt)  

    # M paths GBM
    stock_paths = np.zeros((M, n + 1))  # init St matrix
    stock_paths[:, 0] = S0  # init stock price

    for t in range(1, n + 1):

        # update St for every path
        stock_paths[:, t] = stock_paths[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * norm.rvs(size=M))

    # init payoff matrix
    payoffs = np.maximum(K - stock_paths, 0)  # Calculate the payoff for a put option

    # backward idncution
    for t in range(n - 1, 0, -1):
        in_the_money = stock_paths[:, t] < K  # find in the money
        # lin regression for cont value
        regression = LinearRegression().fit(
            stock_paths[in_the_money, t].reshape(-1, 1),
            payoffs[in_the_money, t + 1] * discount_factor
        )
        continuation_value = regression.predict(stock_paths[in_the_money, t].reshape(-1, 1))

        # calc exercise val and decision
        immediate_exercise_value = K - stock_paths[in_the_money, t]
        exercise = immediate_exercise_value > continuation_value
        # update payoffs based on decision
        payoffs[in_the_money, t] = np.where(exercise, immediate_exercise_value, payoffs[in_the_money, t + 1] * discount_factor)
        # update payoffs for OOTM
        payoffs[~in_the_money, t] = payoffs[~in_the_money, t + 1] * discount_factor

    # avg and discount
    option_price = np.mean(payoffs[:, 1]) * discount_factor
    return option_price + S0 # ez fix

In [8]:
r = 0.02
sigma = 0.15 
S0 = 100  
K = 100  
T = 10 
n = 10  # steps
M = 100000  # sims

option_price = binomial_tree_option_price(S0, K, r, n, T, sigma)
LSMC_price = LSMC_unit_linked_price(S0, K, r, T, sigma, M, n)
print("Continuous Guarantees")
print("Binomial Tree UL price: ", option_price)
print("LS Monte Carlo UL price: ", LSMC_price)

from utils import price_contract_vectorized # binomial tree for UL contract from week 1
print("Final Guarantee")
print("Binomial Tree UL price: ", price_contract_vectorized(r, sigma, T, K, n, S0)+100)


Continuous Guarantees
Binomial Tree UL price:  111.66681188888981
LS Monte Carlo UL price:  111.48534988363275
Final Guarantee
Binomial Tree UL price:  108.99512289501544


## Binomial Tree and LSMC are quite close, as expected UL with cont. guarantee is more expensive